# Meta ML System Design: Problem Framing & Objectives

**Target Level:** Meta E5 → E6  
**Core Skill:** Converting vague product goals into measurable ML objectives  
**Interview Focus:** Demonstrating product-ML alignment thinking

---

## Why This Matters

The #1 differentiator between average and exceptional ML engineers at Meta:

**Average:** "We need to predict clicks"

**Exceptional:** "We need to predict clicks, but optimizing only for clicks leads to clickbait. We should predict a weighted combination of click + dwell time + positive actions - hide - report, where the weights reflect our product values around meaningful engagement."

This notebook teaches you to think like the second engineer.

---

## 1. The Problem Framing Framework

Every ML system design starts with this question:

**"What are we actually trying to optimize?"**

At Meta, this requires translating between three levels:

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                          PROBLEM FRAMING LEVELS                             │
└─────────────────────────────────────────────────────────────────────────────┘

Level 1: BUSINESS OBJECTIVE
├── "Increase user engagement on Facebook"
├── "Grow Instagram Reels watch time"
└── "Improve advertiser ROI"
         │
         │ Translation needed: What does "engagement" mean exactly?
         ▼
Level 2: PRODUCT METRIC
├── Daily Active Users (DAU)
├── Time Spent per Session
├── Posts Created per User
└── Ad Revenue per User
         │
         │ Translation needed: What action causes this metric to move?
         ▼
Level 3: ML OBJECTIVE
├── Predict P(click | user, post)
├── Predict P(watch > 10s | user, video)
├── Predict P(create_post | user, prompt)
└── Predict P(convert | user, ad)
```

> **🧠 Meta Interview Signal**
>
> **Strong E5 Answer:** Clearly articulates all three levels for a given problem. Shows awareness that business objectives need translation.
>
> **E6 Answer Adds:** Discusses the translation gaps—why a product metric might not perfectly align with business objectives (Goodhart's Law), and how ML objectives can game product metrics.
>
> **Common Pitfall:** Jumping straight to "predict clicks" without explaining why clicks matter and what happens when you over-optimize for them.

## 2. Good vs Bad Problem Framing: Examples

### Example 1: News Feed Ranking

| Aspect | ❌ Bad Framing | ✅ Good Framing |
|--------|---------------|----------------|
| **Goal** | "Predict what posts users will click" | "Predict what posts lead to meaningful engagement" |
| **ML Task** | Binary classification: click/no-click | Multi-task: P(click), P(like), P(comment), P(share), P(hide), P(report) |
| **Label** | Click happened | Multiple signals with different weights |
| **Problem** | Optimizes for clickbait | Balances engagement quality |
| **Failure Mode** | Users click but regret it, leave platform | Considered in negative signals (hide, report) |

### Example 2: Friend Suggestions (People You May Know)

| Aspect | ❌ Bad Framing | ✅ Good Framing |
|--------|---------------|----------------|
| **Goal** | "Predict friend request probability" | "Predict mutual friendship probability" |
| **ML Task** | P(send_request \| suggestion) | P(accepted_friendship \| suggestion) |
| **Label** | Request sent | Request sent AND accepted |
| **Problem** | Suggests people who get many requests | Suggests people likely to form real connections |
| **Failure Mode** | Spam popular users, creepy suggestions | Considered: only suggest likely mutual connections |

### Example 3: Notification Ranking

| Aspect | ❌ Bad Framing | ✅ Good Framing |
|--------|---------------|----------------|
| **Goal** | "Maximize notification opens" | "Maximize opens without causing notification fatigue" |
| **ML Task** | P(open \| notification) | P(open) - λ × P(disable_notifications) |
| **Label** | Notification opened | Open and long-term notification preferences |
| **Problem** | Send too many notifications | Balances short-term engagement with retention |
| **Failure Mode** | Users disable notifications or uninstall | Considered in penalty term |

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Demonstration: Why single-objective optimization fails

np.random.seed(42)

# Simulate 1000 posts with different characteristics
n_posts = 1000

# Post types:
# - Clickbait: High click rate, low dwell time, high hide rate
# - Quality: Medium click rate, high dwell time, low hide rate
# - Boring: Low click rate, medium dwell time, low hide rate

post_types = np.random.choice(['clickbait', 'quality', 'boring'], n_posts, p=[0.2, 0.3, 0.5])

def generate_engagement(post_type):
    if post_type == 'clickbait':
        return {
            'p_click': np.random.beta(8, 3),      # High clicks
            'dwell_time': np.random.exponential(5),  # Low dwell
            'p_like': np.random.beta(1, 10),      # Low likes
            'p_hide': np.random.beta(4, 6),       # High hides
        }
    elif post_type == 'quality':
        return {
            'p_click': np.random.beta(4, 4),      # Medium clicks
            'dwell_time': np.random.exponential(30), # High dwell
            'p_like': np.random.beta(4, 4),       # Good likes
            'p_hide': np.random.beta(1, 15),      # Low hides
        }
    else:  # boring
        return {
            'p_click': np.random.beta(2, 8),      # Low clicks
            'dwell_time': np.random.exponential(15), # Medium dwell
            'p_like': np.random.beta(2, 8),       # Low-medium likes
            'p_hide': np.random.beta(1, 20),      # Very low hides
        }

posts = pd.DataFrame([generate_engagement(t) for t in post_types])
posts['type'] = post_types

# Strategy 1: Rank by P(click) only
posts['score_click_only'] = posts['p_click']

# Strategy 2: Rank by composite score
posts['score_composite'] = (
    0.3 * posts['p_click'] + 
    0.01 * posts['dwell_time'] +  # Normalized by typical dwell
    0.3 * posts['p_like'] - 
    0.5 * posts['p_hide']
)

# Compare what gets ranked in top 100 for each strategy
top_100_click = posts.nlargest(100, 'score_click_only')
top_100_composite = posts.nlargest(100, 'score_composite')

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Composition of top 100 by strategy
for ax, top_posts, title in [(axes[0], top_100_click, 'Click-Only Ranking'),
                              (axes[1], top_100_composite, 'Composite Ranking')]:
    type_counts = top_posts['type'].value_counts()
    colors = {'clickbait': 'red', 'quality': 'green', 'boring': 'gray'}
    ax.bar(type_counts.index, type_counts.values, 
           color=[colors[t] for t in type_counts.index])
    ax.set_ylabel('Count in Top 100')
    ax.set_title(title)
    ax.set_ylim(0, 80)

plt.tight_layout()
plt.show()

print("\nTop 100 composition:")
print(f"\nClick-only: {dict(top_100_click['type'].value_counts())}")
print(f"Composite:  {dict(top_100_composite['type'].value_counts())}")

## 3. The Multi-Objective Value Function

At Meta, the ranking score is almost never a single prediction. It's a **value function** that combines multiple predictions:

```
Value = Σ (wᵢ × Pᵢ) for positive actions
      - Σ (wⱼ × Pⱼ) for negative actions
      
Where:
- Pᵢ = model prediction for action i
- wᵢ = product-defined weight for action i
```

### Feed Ranking Example:
```
Value = w_click × P(click)
      + w_like × P(like)
      + w_comment × P(comment)
      + w_share × P(share)
      + w_long_view × P(watch > 10s)
      - w_hide × P(hide)
      - w_report × P(report)
      - w_unfollow × P(unfollow_author)
```

### Why Separate Models from Weights?

1. **Model changes require retraining** (days/weeks)
2. **Weight changes are instant** (no retraining)
3. **Product can tune weights** without ML team
4. **A/B test weights** independently from model quality

In [ ]:
# Demonstration: Tuning weights to change behavior

# Suppose we have a model that predicts these probabilities
np.random.seed(42)
n_posts = 500

predictions = pd.DataFrame({
    'post_id': range(n_posts),
    'p_click': np.random.beta(3, 5, n_posts),
    'p_like': np.random.beta(2, 8, n_posts),
    'p_comment': np.random.beta(1, 15, n_posts),
    'p_share': np.random.beta(1, 20, n_posts),
    'p_hide': np.random.beta(1, 30, n_posts),
    'is_video': np.random.binomial(1, 0.3, n_posts),  # 30% are videos
})

# Different weight configurations for different product goals
weight_configs = {
    'Engagement Focus': {
        'click': 1.0, 'like': 0.5, 'comment': 0.5, 'share': 0.5, 'hide': 2.0
    },
    'Quality Focus': {
        'click': 0.2, 'like': 1.0, 'comment': 2.0, 'share': 2.0, 'hide': 5.0
    },
    'Video Push': {
        'click': 0.5, 'like': 0.5, 'comment': 0.5, 'share': 0.5, 'hide': 2.0,
        'video_boost': 3.0  # Extra boost for videos
    },
}

def compute_value(df, weights):
    score = (
        weights['click'] * df['p_click'] +
        weights['like'] * df['p_like'] +
        weights['comment'] * df['p_comment'] +
        weights['share'] * df['p_share'] -
        weights['hide'] * df['p_hide']
    )
    if 'video_boost' in weights:
        score += weights['video_boost'] * df['is_video']
    return score

# Compute scores under each config
for name, weights in weight_configs.items():
    predictions[f'score_{name}'] = compute_value(predictions, weights)

# Show how top 10 differs
print("Top 10 posts under different objectives:\n")
for name in weight_configs:
    top_10 = predictions.nlargest(10, f'score_{name}')
    video_pct = top_10['is_video'].mean() * 100
    avg_hide = top_10['p_hide'].mean()
    avg_share = top_10['p_share'].mean()
    print(f"{name}:")
    print(f"  - {video_pct:.0f}% videos in top 10")
    print(f"  - Avg P(hide): {avg_hide:.4f}")
    print(f"  - Avg P(share): {avg_share:.4f}\n")

> **🧠 Meta Interview Signal**
>
> **Strong E5 Answer:** Explains the value function concept and why it's useful. Shows awareness that weights are product decisions.
>
> **E6 Answer Adds:** Discusses how to tune weights (grid search with A/B tests), potential gaming issues (if comment weight is high, people may leave low-quality comments), and governance around weight changes.
>
> **Common Pitfall:** Trying to learn the weights end-to-end. While possible, it removes product control and makes the system a black box.

## 4. Proxy Metrics: When You Can't Measure What You Want

Often, what you **want** to optimize is not directly measurable:

| What We Want | Why It's Hard | Proxy We Use |
|--------------|--------------|-------------|
| User satisfaction | Subjective, expensive to measure | Engagement signals + surveys |
| Content quality | No universal definition | Shares, comments, dwell time |
| Long-term retention | Takes months to observe | Session-level engagement |
| Ad effectiveness | Delayed conversions | Click + estimated conversion |

### The Proxy Problem

```
What we want:        User finds valuable content
                            ↑
                     (not measurable)
                            |
What we measure:     User clicks on content
                            ↑
                     (correlation, not causation)
                            |
What we optimize:    P(click | user, content)
```

**Goodhart's Law:** "When a measure becomes a target, it ceases to be a good measure."

If we optimize too hard for clicks, content creators learn to game clicks (clickbait), and clicks no longer indicate value.

In [ ]:
# Demonstration: Proxy metric divergence over time

def simulate_proxy_divergence(n_rounds=50, optimization_strength=0.1):
    """
    Simulate what happens when we optimize a proxy metric.
    
    Over time, content creators learn to game the proxy,
    and correlation between proxy and true objective decreases.
    """
    np.random.seed(42)
    
    # Initial state: proxy (clicks) correlates with true objective (satisfaction)
    correlation = 0.8
    
    # Gaming pressure: how much creators game the metric each round
    gaming_pressure = 0.0
    
    history = []
    
    for round_num in range(n_rounds):
        # As we optimize for proxy, creators learn to game it
        gaming_pressure += optimization_strength * (1 - gaming_pressure)
        
        # Gaming reduces correlation between proxy and true objective
        correlation = 0.8 * (1 - gaming_pressure) + 0.1 * gaming_pressure
        
        # Proxy metric goes up (we're optimizing it!)
        proxy_metric = 1 + round_num * 0.02
        
        # True objective may not follow
        true_objective = 1 + round_num * 0.02 * correlation
        
        history.append({
            'round': round_num,
            'proxy_metric': proxy_metric,
            'true_objective': true_objective,
            'correlation': correlation,
            'gaming_pressure': gaming_pressure
        })
    
    return pd.DataFrame(history)

# Compare weak vs strong optimization
weak_opt = simulate_proxy_divergence(optimization_strength=0.02)
strong_opt = simulate_proxy_divergence(optimization_strength=0.1)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Plot metrics over time
ax = axes[0]
ax.plot(strong_opt['round'], strong_opt['proxy_metric'], 'b-', 
        label='Proxy (Clicks)', linewidth=2)
ax.plot(strong_opt['round'], strong_opt['true_objective'], 'g--', 
        label='True Objective (Satisfaction)', linewidth=2)
ax.fill_between(strong_opt['round'], 
                strong_opt['true_objective'], 
                strong_opt['proxy_metric'],
                alpha=0.3, color='red', label='Gap (Gaming)')
ax.set_xlabel('Optimization Rounds')
ax.set_ylabel('Metric Value')
ax.set_title('Strong Optimization: Proxy Diverges from Truth')
ax.legend()

# Plot correlation decay
ax = axes[1]
ax.plot(weak_opt['round'], weak_opt['correlation'], 'g-', 
        label='Weak optimization', linewidth=2)
ax.plot(strong_opt['round'], strong_opt['correlation'], 'r-', 
        label='Strong optimization', linewidth=2)
ax.set_xlabel('Optimization Rounds')
ax.set_ylabel('Correlation (Proxy ↔ True Objective)')
ax.set_title('Correlation Decay Due to Gaming')
ax.legend()
ax.set_ylim(0, 1)

plt.tight_layout()
plt.show()

### Solutions to Proxy Divergence

1. **Multiple proxies:** Combine several signals (clicks + dwell + likes - hides)
2. **Negative signals:** Include "regret" signals (hide, unfollow, report)
3. **Long-term outcomes:** Periodically check against retention, survey satisfaction
4. **Regularization:** Don't optimize too hard; keep some exploration
5. **Human review:** Periodic audits of top-ranked content

## 5. Position Bias and Selection Bias

When framing problems, you must account for biases in your labels:

### Position Bias

Items shown in position 1 get more clicks than identical items in position 10, **regardless of relevance**.

```
Position    Observed CTR    True Relevance
   1           15%              Medium
   2           8%               High      ← Better item, worse position
   3           5%               High  
   4           3%               Low
   ...
```

If you train on raw click data, you learn that "position 1 items are good," not that "relevant items are good."

### Selection Bias

You only observe outcomes for items you showed. Items never shown have no labels.

```
Candidate Pool: [A, B, C, D, E, F, G, H, I, J]
                 │  │  │                      
Shown to user:  [A, B, C]  ← Only these get labels!
                 │  │  │
User clicked:   [A]       ← Training signal

Result: We never learn if D, E, F, G, H, I, J were good.
```

In [ ]:
# Demonstration: Position bias in click data

np.random.seed(42)

# Simulate 1000 impressions with position bias
n_impressions = 5000

# True relevance (what we want to learn)
true_relevance = np.random.uniform(0, 1, n_impressions)

# Position (1-10)
position = np.random.randint(1, 11, n_impressions)

# Position bias: higher positions get more clicks regardless of relevance
position_bias = 1 / (position ** 0.5)  # Position 1 = 1.0, Position 10 = 0.32

# Observed click probability = relevance × position_bias
click_prob = true_relevance * position_bias
clicks = np.random.binomial(1, np.clip(click_prob, 0, 1))

data = pd.DataFrame({
    'true_relevance': true_relevance,
    'position': position,
    'position_bias': position_bias,
    'click': clicks
})

# Naive model: predict click from relevance only (ignoring position)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Model 1: Naive (just relevance as feature, trained on biased clicks)
X_naive = data[['true_relevance']].values
model_naive = LogisticRegression()
model_naive.fit(X_naive, clicks)

# Model 2: Position-aware (include position as feature)
X_position = data[['true_relevance', 'position']].values
model_position = LogisticRegression()
model_position.fit(X_position, clicks)

# Evaluate: which model better captures TRUE relevance?
# Test on position-1 data only (where bias is constant)
pos1_mask = data['position'] == 1

print("Correlation with TRUE relevance (on position-1 data):")
print(f"  Naive model:    {np.corrcoef(model_naive.predict_proba(X_naive[pos1_mask])[:, 1], data.loc[pos1_mask, 'true_relevance'])[0,1]:.3f}")
print(f"  Position-aware: {np.corrcoef(model_position.predict_proba(X_position[pos1_mask])[:, 1], data.loc[pos1_mask, 'true_relevance'])[0,1]:.3f}")

# Show average click rate by position
print("\nObserved CTR by position (demonstrates bias):")
ctr_by_pos = data.groupby('position')['click'].mean()
for pos in range(1, 6):
    print(f"  Position {pos}: {ctr_by_pos[pos]:.1%}")

### Handling Position Bias

1. **Include position as feature:** Let model learn the bias, then set position=1 at inference
2. **Inverse propensity weighting:** Weight examples by 1/P(shown at this position)
3. **Randomization:** Randomly shuffle some traffic to get unbiased labels
4. **Multi-task learning:** Predict (relevance, position_ctr) separately

> **🧠 Meta Interview Signal**
>
> **Strong E5 Answer:** Identifies position bias as a problem and proposes at least one solution.
>
> **E6 Answer Adds:** Discusses tradeoffs between solutions (randomization costs engagement, IPW has high variance), and how to validate debiasing worked.
>
> **Common Pitfall:** Training directly on click data without mentioning position bias at all.

## 6. Framing for Different Meta Products

### Feed Ranking
```
Business Goal: Keep users engaged with meaningful content
Product Metric: Daily engagement, time spent, content creation
ML Objective: Score(post) = Σ wᵢ × P(actionᵢ | user, post)
Key Actions: click, like, comment, share, save, long_view, hide, report
```

### Ads Ranking
```
Business Goal: Maximize revenue while maintaining user experience
Product Metric: Revenue, advertiser ROI, user ad tolerance
ML Objective: Score(ad) = Bid × P(click) × P(convert | click) × Quality
Key Considerations: Ad load (how many ads), relevance, diversity
```

### Reels Recommendations
```
Business Goal: Maximize watch time and creator ecosystem
Product Metric: Watch time, completion rate, creator uploads
ML Objective: Score(reel) = P(watch > threshold) × expected_watch_time
Key Considerations: Novelty, diversity, cold start for new creators
```

### Notifications
```
Business Goal: Re-engage users without causing fatigue
Product Metric: Opens, DAU lift, unsubscribe rate
ML Objective: Score(notif) = P(open) × value - λ × P(disable)
Key Considerations: Timing, frequency capping, notification type priority
```

In [ ]:
# Interactive exercise: Frame a new problem

def problem_framing_exercise():
    """
    Given a vague product goal, practice translating it to ML objective.
    """
    problems = [
        {
            "product_goal": "Reduce harmful content on the platform",
            "good_framing": {
                "ml_objective": "P(violates_policy | content) as ranking penalty",
                "labels": "Human review decisions, user reports, policy violations",
                "tradeoffs": "False positives hurt creators, false negatives hurt users",
                "multi_stage": "Fast classifier for all content → deep model for borderline"
            }
        },
        {
            "product_goal": "Help users discover new interests",
            "good_framing": {
                "ml_objective": "P(engage | user, topic) for topics user hasn't seen",
                "labels": "Engagement on content outside user's usual topics",
                "tradeoffs": "Exploration hurts short-term engagement, helps long-term",
                "multi_stage": "Topic model → user interest prediction → candidate ranking"
            }
        },
        {
            "product_goal": "Improve Marketplace trust and safety",
            "good_framing": {
                "ml_objective": "P(scam | listing) + P(successful_transaction | listing)",
                "labels": "Reported scams, completed transactions, returns/disputes",
                "tradeoffs": "Blocking scams vs. blocking legitimate sellers",
                "multi_stage": "Listing classifier → seller reputation → buyer-seller match"
            }
        }
    ]
    
    print("=" * 70)
    print("PROBLEM FRAMING EXERCISE")
    print("=" * 70)
    
    for i, problem in enumerate(problems, 1):
        print(f"\n{'─' * 70}")
        print(f"Problem {i}: {problem['product_goal']}")
        print(f"{'─' * 70}")
        print("\n📋 Good Framing:")
        for key, value in problem['good_framing'].items():
            print(f"   • {key}: {value}")

problem_framing_exercise()

---

## 🎯 Interview Drills

### Drill 1: "How would you frame the objective for [X product]?"

**Framework for answering:**
1. State the business objective clearly
2. Identify measurable proxy metrics
3. Define the ML task (what are you predicting?)
4. List positive and negative signals
5. Describe the value function
6. Acknowledge tradeoffs and failure modes

**Example answer for "Instagram Explore page":**

> "The business objective is helping users discover content they'll love, which drives engagement and retention. The ML task is ranking candidate posts by expected value. Value is a weighted combination of P(save), P(like), P(follow_author), P(long_view) minus P(hide) and P(report). The weights prioritize saves and follows because they indicate genuine interest, not just casual consumption. Failure modes include filter bubbles—if we only show similar content, users get bored. We'd need exploration and diversity constraints in re-ranking."

---

### Drill 2: "What's wrong with just predicting clicks?"

**Strong answer:**
> "Clicks are easy to game with clickbait—sensational headlines, misleading thumbnails. Clicks don't distinguish between satisfied users and frustrated users who clicked but immediately left. We should also predict dwell time, positive actions (like, comment, share), and negative signals (hide, report). The value function should penalize content that gets clicks but causes regret."

---

### Drill 3: "How would you handle position bias?"

**Strong answer:**
> "Position bias means items in position 1 get more clicks regardless of relevance. Three solutions: (1) Include position as a feature during training, then set position=1 at inference to get unbiased scores. (2) Use inverse propensity weighting—weight training examples by 1/P(shown at this position). (3) Randomize a small percentage of traffic to collect unbiased labels. Each has tradeoffs: randomization hurts short-term metrics, IPW has high variance for rare positions, and the feature approach assumes position bias is constant across users."

---

### Drill 4: "How do you know your proxy metric is still valid?"

**Strong answer:**
> "Proxy metrics degrade over time as the system optimizes for them and creators learn to game them. We validate by: (1) Periodic user surveys asking about satisfaction, (2) Checking correlation with long-term outcomes like retention, (3) Human review of top-ranked content, (4) Monitoring for divergence—if clicks go up but time spent or returns go down, the proxy is breaking. When proxy-truth correlation drops, we add new signals or regularize optimization."

---

## Summary: Problem Framing Checklist

Before building any ML system, verify:

- [ ] **Business objective is clear:** What does success look like?
- [ ] **Product metrics identified:** How will we measure success?
- [ ] **ML task defined:** What exactly are we predicting?
- [ ] **Labels available:** Where do labels come from? Any bias?
- [ ] **Positive signals listed:** What user actions indicate value?
- [ ] **Negative signals listed:** What actions indicate harm?
- [ ] **Value function designed:** How do we combine predictions?
- [ ] **Failure modes identified:** What could go wrong?
- [ ] **Proxy validity checked:** Will this proxy still work in 6 months?
- [ ] **Position/selection bias addressed:** How do we debias labels?